In [1]:
from google.colab import drive

drive.mount('/content/drive')
path = "/content/drive/My Drive/Datamining/TextMining/"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!cp "/content/drive/My Drive/Datamining/TextMining/Notebooks/functions.py" .
import functions as f
import importlib
importlib.reload(f)
f.v()

2020-06-02 07:31


In [0]:
import pandas as pd
import io
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
import gc
import csv

# Carga de datos y limpieza del dataset

In [4]:
df = f.loadDatasets(path + "Dataset/")
df

,date,text,language,language_proba
0,2019-05-10,share,-,0.000000
1,2019-05-10,😂😂,-,0.000000
2,2019-05-10,@mark_dow Bitcoin is the one imv,en,0.980272
3,2019-05-10,Bitcoin FUDsters RN https://t.co/Yxo3TCxQci,en,0.449887
4,2019-05-10,"@KennETHbosak ""Banking on Bitcoin"" on Netflix.",en,0.685419
...,...,...,...,...
11867881,2019-11-23,Happy #FibonacciDay A while back I created a...,en,0.939878
11867882,2019-11-23,Bitcoin Suisse Certificates :) https://t.co/nd...,en,0.457228
11867883,2019-11-23,Register now for the early access of the Codug...,en,0.884063
11867884,2019-11-23,@btc @btc Do you know that BTC Baskets isn't...,en,0.925460


In [5]:
# Nos quedamos solamente con inglés
df = df[df['language'] == 'en']
df

,date,text,language,language_proba
2,2019-05-10,@mark_dow Bitcoin is the one imv,en,0.980272
3,2019-05-10,Bitcoin FUDsters RN https://t.co/Yxo3TCxQci,en,0.449887
4,2019-05-10,"@KennETHbosak ""Banking on Bitcoin"" on Netflix.",en,0.685419
11,2019-05-11,$1000 left!,en,0.247924
14,2019-05-11,Never too late to buy. The best timing to buy ...,en,0.937032
...,...,...,...,...
11867881,2019-11-23,Happy #FibonacciDay A while back I created a...,en,0.939878
11867882,2019-11-23,Bitcoin Suisse Certificates :) https://t.co/nd...,en,0.457228
11867883,2019-11-23,Register now for the early access of the Codug...,en,0.884063
11867884,2019-11-23,@btc @btc Do you know that BTC Baskets isn't...,en,0.925460


In [0]:
df.drop(['language'], axis=1, inplace=True)
# Después vamos a usar el mes para poder armar los distintos datasets de train, validación y test
df['month'] = df.date.apply(lambda date: f.getMonth(date))
df

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,date,text,language_proba,month
2,2019-05-10,@mark_dow Bitcoin is the one imv,0.980272,5
3,2019-05-10,Bitcoin FUDsters RN https://t.co/Yxo3TCxQci,0.449887,5
4,2019-05-10,"@KennETHbosak ""Banking on Bitcoin"" on Netflix.",0.685419,5
11,2019-05-11,$1000 left!,0.247924,5
14,2019-05-11,Never too late to buy. The best timing to buy ...,0.937032,5
...,...,...,...,...
11867881,2019-11-23,Happy #FibonacciDay A while back I created a...,0.939878,11
11867882,2019-11-23,Bitcoin Suisse Certificates :) https://t.co/nd...,0.457228,11
11867883,2019-11-23,Register now for the early access of the Codug...,0.884063,11
11867884,2019-11-23,@btc @btc Do you know that BTC Baskets isn't...,0.925460,11


In [0]:
# Threshold en 0, para que tenga en cuenta todos.
# Así después podemos usar la columna de la proba del lenguaje para crear otra columna bool.
df = f.addIsSpamColumn(df, languageProbaThreshold = 0.0)
# Le cambiamos el nombre porque la de is_spam la vamos a asignar con la RNN
df.rename(columns={'is_spam': 'is_repeated'}, inplace=True)
df

,date,text,language_proba,month,is_repeated
0,2019-05-10,@mark_dow Bitcoin is the one imv,0.980272,5,False
1,2019-05-10,Bitcoin FUDsters RN https://t.co/Yxo3TCxQci,0.449887,5,False
2,2019-05-10,"@KennETHbosak ""Banking on Bitcoin"" on Netflix.",0.685419,5,False
3,2019-05-11,$1000 left!,0.247924,5,False
4,2019-05-11,Never too late to buy. The best timing to buy ...,0.937032,5,False
...,...,...,...,...,...
10329768,2019-11-23,Happy #FibonacciDay A while back I created a...,0.939878,11,False
10329769,2019-11-23,Bitcoin Suisse Certificates :) https://t.co/nd...,0.457228,11,False
10329770,2019-11-23,Register now for the early access of the Codug...,0.884063,11,False
10329771,2019-11-23,@btc @btc Do you know that BTC Baskets isn't...,0.925460,11,False


# Carga de los modelos y config inicial

In [0]:
max_length = 50
trunc_type='post'
padding_type='post'

In [0]:
# Leemos el tokenizer que usamos originalmente para entrenar los modelos
with io.open(path + 'Modelos/' + 'tokenizer.json', 'r', encoding='utf-8') as json_file:
    tokenizer = tf.keras.preprocessing.text.tokenizer_from_json(json_file.readline())

In [0]:
spam_model = tf.keras.models.load_model(path + 'Modelos/' + 'spam_bigru_pre_2020-05-30_2120')
spam_model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 50, 300)           3000300   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 50, 256)           330240    
_________________________________________________________________
dropout_2 (Dropout)          (None, 50, 256)           0         
_________________________________________________________________
bidirectional_3 (Bidirection (None, 256)               296448    
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 10)                2570      
_________________________________________________________________
dense_10 (Dense)             (None, 10)               

In [0]:
news_model = tf.keras.models.load_model(path + 'Modelos/' + 'news_gru_pre_2020-05-30_2323')
news_model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 50, 300)           3000300   
_________________________________________________________________
gru_6 (GRU)                  (None, 128)               165120    
_________________________________________________________________
dense_12 (Dense)             (None, 10)                1290      
_________________________________________________________________
dense_13 (Dense)             (None, 10)                110       
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 11        
Total params: 3,166,831
Trainable params: 3,166,831
Non-trainable params: 0
_________________________________________________________________


# Generación de columnas is_spam y is_news para cada mes

In [0]:
for i_month in range(5, 12):
    df_month = df[df['month'] == i_month]

    # Forzamos que corra el garbage para no quedarnos sin memoria (pasaba todo el tiempo cantes de correrlo mes a mes)
    print('Starting ', i_month, 'gc.collect()', gc.collect(), gc.collect(), gc.collect())
    
    # Preparamos los datos como los esperan las RNN
    x_text = df_month['text'].apply(f.normalizeForTokenization)
    x_text = tokenizer.texts_to_sequences(x_text)
    x_text = pad_sequences(x_text, maxlen=max_length, padding=padding_type, truncating=trunc_type)
    
    # Forzamos que corra el garbage para no quedarnos sin memoria (pasaba todo el tiempo cantes de correrlo mes a mes)
    print(i_month, 'gc.collect()', gc.collect(), gc.collect(), gc.collect())

    # Predecimos is_spam
    pred = spam_model.predict(x_text, batch_size=1000, verbose=1)
    pred = pred[:,0] > 0.5
    df_month['is_spam'] = pred

    # Predecimos is_news
    pred = news_model.predict(x_text, batch_size=1000, verbose=1)
    pred = pred[:,0] > 0.5
    df_month['is_news'] = pred

    # Guardamos el dataframe del mes actual
    df_month.to_csv(f'{path}Dataset/v1/{str(i_month).zfill(2)}-tagged.csv.gz',
                    index = False, compression='gzip', escapechar='\\', quotechar='"', encoding = "utf-8",
                    quoting=csv.QUOTE_NONNUMERIC, sep=";")


Starting  5 gc.collect() 148 0 0
5 gc.collect() 0 0 0
1276/1276 [==============================] - 106s 83ms/step


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


1276/1276 [==============================] - 45s 35ms/step


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Starting  6 gc.collect() 1346 0 0
6 gc.collect() 0 0 0
2002/2002 [==============================] - 68s 34ms/step
Starting  7 gc.collect() 675 0 0
7 gc.collect() 0 0 0
1781/1781 [==============================] - 62s 35ms/step
Starting  8 gc.collect() 675 0 0
8 gc.collect() 0 0 0
1488/1488 [==============================] - 52s 35ms/step
Starting  9 gc.collect() 675 0 0
9 gc.collect() 0 0 0
1481/1481 [==============================] - 52s 35ms/step
Starting  10 gc.collect() 675 0 0
10 gc.collect() 0 0 0
1359/1359 [==============================] - 48s 35ms/step
Starting  11 gc.collect() 675 0 0
11 gc.collect() 0 0 0
947/947 [==============================] - 32s 34ms/step


In [0]:
# Pruebo que se puedan cargar algunos meses de los taggeados
df_tagged = f.loadTaggedDatasets(path + "Dataset/", months=[5, 6])
df_tagged

,date,text,language_proba,month,is_repeated,is_spam,is_news
0,2019-05-10,@mark_dow Bitcoin is the one imv,0.980272,5,False,False,False
1,2019-05-10,Bitcoin FUDsters RN https://t.co/Yxo3TCxQci,0.449887,5,False,False,True
2,2019-05-10,"@KennETHbosak ""Banking on Bitcoin"" on Netflix.",0.685419,5,False,False,False
3,2019-05-11,$1000 left!,0.247924,5,False,False,False
4,2019-05-11,Never too late to buy. The best timing to buy ...,0.937032,5,False,False,False
...,...,...,...,...,...,...,...
3276320,2019-06-27,Hello Bitsler Family! How are you ? 😍 These W...,0.817517,6,False,True,False
3276321,2019-06-29,"In an effort to create trust in the platform, ...",0.687963,6,False,True,False
3276322,2019-06-28,As additional landlords and tenants register w...,0.906536,6,False,True,False
3276323,2019-06-30,@xWinter_Maix All my profit from bitcoin tribu...,0.529841,6,False,True,True


In [0]:
(df_tagged.month.unique(), df_tagged.is_repeated.unique(), df_tagged.is_spam.unique(), df_tagged.is_news.unique())

(array([5, 6]),
 array([False,  True]),
 array([False,  True]),
 array([False,  True]))